# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8ccc11d2b0>
├── 'a' --> tensor([[ 0.6756,  0.0030,  1.2495],
│                   [-0.3547, -0.8126, -0.4596]])
└── 'x' --> <FastTreeValue 0x7f8cf04957f0>
    └── 'c' --> tensor([[ 2.1625e+00, -4.1443e-03, -3.7981e-02,  9.4282e-01],
                        [ 7.3548e-02, -6.5365e-01, -2.9493e-05,  2.0976e-01],
                        [-7.3438e-01, -1.0604e+00,  8.5125e-01, -8.7493e-01]])

In [4]:
t.a

tensor([[ 0.6756,  0.0030,  1.2495],
        [-0.3547, -0.8126, -0.4596]])

In [5]:
%timeit t.a

61.4 ns ± 0.0548 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8ccc11d2b0>
├── 'a' --> tensor([[ 0.7574,  1.3851,  0.2728],
│                   [-1.1601, -0.9871,  1.2608]])
└── 'x' --> <FastTreeValue 0x7f8cf04957f0>
    └── 'c' --> tensor([[ 2.1625e+00, -4.1443e-03, -3.7981e-02,  9.4282e-01],
                        [ 7.3548e-02, -6.5365e-01, -2.9493e-05,  2.0976e-01],
                        [-7.3438e-01, -1.0604e+00,  8.5125e-01, -8.7493e-01]])

In [7]:
%timeit t.a = new_value

60.2 ns ± 0.021 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6756,  0.0030,  1.2495],
               [-0.3547, -0.8126, -0.4596]]),
    x: Batch(
           c: tensor([[ 2.1625e+00, -4.1443e-03, -3.7981e-02,  9.4282e-01],
                      [ 7.3548e-02, -6.5365e-01, -2.9493e-05,  2.0976e-01],
                      [-7.3438e-01, -1.0604e+00,  8.5125e-01, -8.7493e-01]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6756,  0.0030,  1.2495],
        [-0.3547, -0.8126, -0.4596]])

In [11]:
%timeit b.a

57.3 ns ± 0.0617 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.8544, -0.6987, -0.6787],
               [-1.0340, -0.2523,  0.4787]]),
    x: Batch(
           c: tensor([[ 2.1625e+00, -4.1443e-03, -3.7981e-02,  9.4282e-01],
                      [ 7.3548e-02, -6.5365e-01, -2.9493e-05,  2.0976e-01],
                      [-7.3438e-01, -1.0604e+00,  8.5125e-01, -8.7493e-01]]),
       ),
)

In [13]:
%timeit b.a = new_value

478 ns ± 0.0812 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

825 ns ± 0.134 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 23.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 930 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 618 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f8c387628b0>
├── 'a' --> tensor([[[ 0.6756,  0.0030,  1.2495],
│                    [-0.3547, -0.8126, -0.4596]],
│           
│                   [[ 0.6756,  0.0030,  1.2495],
│                    [-0.3547, -0.8126, -0.4596]],
│           
│                   [[ 0.6756,  0.0030,  1.2495],
│                    [-0.3547, -0.8126, -0.4596]],
│           
│                   [[ 0.6756,  0.0030,  1.2495],
│                    [-0.3547, -0.8126, -0.4596]],
│           
│                   [[ 0.6756,  0.0030,  1.2495],
│                    [-0.3547, -0.8126, -0.4596]],
│           
│                   [[ 0.6756,  0.0030,  1.2495],
│                    [-0.3547, -0.8126, -0.4596]],
│           
│                   [[ 0.6756,  0.0030,  1.2495],
│                    [-0.3547, -0.8126, -0.4596]],
│           
│                   [[ 0.6756,  0.0030,  1.2495],
│                    [-0.3547, -0.8126, -0.4596]]])
└── 'x' --> <FastTreeValue 0x7f8c38762a90>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 52.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f8c38788f40>
├── 'a' --> tensor([[ 0.6756,  0.0030,  1.2495],
│                   [-0.3547, -0.8126, -0.4596],
│                   [ 0.6756,  0.0030,  1.2495],
│                   [-0.3547, -0.8126, -0.4596],
│                   [ 0.6756,  0.0030,  1.2495],
│                   [-0.3547, -0.8126, -0.4596],
│                   [ 0.6756,  0.0030,  1.2495],
│                   [-0.3547, -0.8126, -0.4596],
│                   [ 0.6756,  0.0030,  1.2495],
│                   [-0.3547, -0.8126, -0.4596],
│                   [ 0.6756,  0.0030,  1.2495],
│                   [-0.3547, -0.8126, -0.4596],
│                   [ 0.6756,  0.0030,  1.2495],
│                   [-0.3547, -0.8126, -0.4596],
│                   [ 0.6756,  0.0030,  1.2495],
│                   [-0.3547, -0.8126, -0.4596]])
└── 'x' --> <FastTreeValue 0x7f8cc4bb0ac0>
    └── 'c' --> tensor([[ 2.1625e+00, -4.1443e-03, -3.7981e-02,  9.4282e-01],
                        [ 7.3548e-02, -6.5365e-01, -2.9493e-05,

In [23]:
%timeit t_cat(trees)

29.9 µs ± 67.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.7 µs ± 36.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 2.1625e+00, -4.1443e-03, -3.7981e-02,  9.4282e-01],
                       [ 7.3548e-02, -6.5365e-01, -2.9493e-05,  2.0976e-01],
                       [-7.3438e-01, -1.0604e+00,  8.5125e-01, -8.7493e-01]],
              
                      [[ 2.1625e+00, -4.1443e-03, -3.7981e-02,  9.4282e-01],
                       [ 7.3548e-02, -6.5365e-01, -2.9493e-05,  2.0976e-01],
                       [-7.3438e-01, -1.0604e+00,  8.5125e-01, -8.7493e-01]],
              
                      [[ 2.1625e+00, -4.1443e-03, -3.7981e-02,  9.4282e-01],
                       [ 7.3548e-02, -6.5365e-01, -2.9493e-05,  2.0976e-01],
                       [-7.3438e-01, -1.0604e+00,  8.5125e-01, -8.7493e-01]],
              
                      [[ 2.1625e+00, -4.1443e-03, -3.7981e-02,  9.4282e-01],
                       [ 7.3548e-02, -6.5365e-01, -2.9493e-05,  2.0976e-01],
                       [-7.3438e-01, -1.0604e+00,  8.5125e-01, -8.7493e-01]],
      

In [26]:
%timeit Batch.stack(batches)

78.4 µs ± 242 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 2.1625e+00, -4.1443e-03, -3.7981e-02,  9.4282e-01],
                      [ 7.3548e-02, -6.5365e-01, -2.9493e-05,  2.0976e-01],
                      [-7.3438e-01, -1.0604e+00,  8.5125e-01, -8.7493e-01],
                      [ 2.1625e+00, -4.1443e-03, -3.7981e-02,  9.4282e-01],
                      [ 7.3548e-02, -6.5365e-01, -2.9493e-05,  2.0976e-01],
                      [-7.3438e-01, -1.0604e+00,  8.5125e-01, -8.7493e-01],
                      [ 2.1625e+00, -4.1443e-03, -3.7981e-02,  9.4282e-01],
                      [ 7.3548e-02, -6.5365e-01, -2.9493e-05,  2.0976e-01],
                      [-7.3438e-01, -1.0604e+00,  8.5125e-01, -8.7493e-01],
                      [ 2.1625e+00, -4.1443e-03, -3.7981e-02,  9.4282e-01],
                      [ 7.3548e-02, -6.5365e-01, -2.9493e-05,  2.0976e-01],
                      [-7.3438e-01, -1.0604e+00,  8.5125e-01, -8.7493e-01],
                      [ 2.1625e+00, -4.1443e-03, -3.7981e-02,  9.42

In [28]:
%timeit Batch.cat(batches)

142 µs ± 182 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

317 µs ± 1.77 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
